In [1]:
import numpy as np
import math
import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
import mayavi
%matplotlib inline

# Partial Trace 

We seek to defin square matrix $M \in L(\mathcal{X}_1 \otimes \mathcal{X}_2 \otimes ... \otimes \mathcal{X}_n)$ can be written 
$$
    M = \sum_{a_1, a_2, ..., a_n} 
$$

In [ ]:
def PartialTrace(Mat, DimList, IndexList):
    """
    Calculates the partial trace of Mat 
    
    """

# Channel definition

# Types of channel representations

Given a channel $\Phi$ acting on square operators $X \in L(\mathcal{X})$ and outputing $Y \in L(\mathcal{Y})$, we may define four types of representations:

Natural representation: the matrix $K(\Phi) \in L(\mathcal{X}^2, \mathcal{Y}^2)$ for which
$$
   K(\Phi) \text{vec}(X) = \text{vec}( \Phi(X) )
$$

Operator-sum representation (Kraus): an operator set $\{A_\alpha : \alpha \in \Sigma \}$ of $A_a \in L(\mathcal{X}, \mathcal{Y})$ such that. 
$$
    \Phi(X) = \sum_{a \in \Sigma} A_a X A_a^\dagger
$$

Partial-trace representation (Stinespring): a rectangular isometry $A \in L(\mathcal{X}, \mathcal{Y} \otimes \mathcal{Z})$
$$
    \Phi(X) = \text{Tr}_{\mathcal{Z}}( A X A^\dagger). 
$$

Dual-state representation (Choi-Jamiolkowski): Choi matrix $J(\Phi) \in L(\mathcal{X}^2, \mathcal{Y}^2)$ 
$$
    J(\Phi) = \sum_{a, b \in \Sigma} |a\rangle \langle b | \otimes  \Phi( |a\rangle \langle b | )  
$$
(this is a right-handed Choi matrix)

---

Remarks: 

In the following, we assume $\Phi$ to be completely positive and trace-preserving. 

---

# From channel to Choi

To determine the choi matrix, we need to calculate $\Phi(e_{a, b})$ for all pairs $(a, b)$. The key equations are 
$$
    \Phi( e_{a, b} ) = \text{mat}( K(\Phi) |a \rangle | b \rangle )
$$
and
$$
    J(\Phi) = \sum_{a, b} e_{a, b} \otimes \Phi(e_{a, b})
$$

In [2]:
def PMat_to_Choi(PMat):
    """
    PMat_to_Choi : ProcessMatrix -> ChoiMatrix 
    
    takes as input an d^2 x d^2 process matrix and outputs a choi matrix. 
    
    we assume here the right-handed Choi representation
    """
    shape = PMat.shape
    BasisOutputs = []
    
    dim = np.sqrt(shape[0])
    # verify that the dimensions of PMat are square numbers
    if(abs(int(dim) - dim) > 0.0001):
        print("ERROR: PMat dims are not square!")
    dim = int(dim)
    
    # convert each row of PMat into a dxd matrix
    for r in range(shape[0]):
        row = PMat[r]
        mat = row.reshape((dim, dim))
        BasisOutputs.append(mat)
    
    # arrange the basis outputs into a 2d list 
    ChoiSet = []
    for d1 in range(dim):
        ChoiSet.append([])
        for d2 in range(dim):
            ChoiSet[d1].append( BasisOutputs[d1*dim+d2] )            
        
    # construct block matrix J(Phi)
    ChoiMat = np.block(ChoiSet)
    return ChoiMat

# From Choi to Kraus

The relevent equations are 

$$
    J(\Phi) = \sum_{a \in \Sigma} \text{vec}( A_a ) \text{vec}( A_a )^\dagger
$$

and 

$$
    J(\Phi) = S \Lambda S^\dagger = \sum_{a = 1}^{\text{rank}(J)} \lambda_a S |a \rangle \langle a| S^\dagger.
$$

So, 
$$
    A_a = \text{mat}(\sqrt{\lambda} S |a \rangle )
$$

In [3]:
def matrize(vector, epsilon=1e-4):
    dim = math.sqrt(vector.size)
    if( abs(dim - int(dim)) > epsilon):
        print("ERROR! in matrize(vector); length is not a square number")
    dim = int(dim)
    
    arry = []
    for d1 in range(dim):
        arry.append([])
        for d2 in range(dim):
            arry[d1].append(vector[d1*dim+d2])
    return np.array(arry)

In [4]:
def Choi_to_Kraus(ChoiMat):
    Lambda, Smat = np.linalg.eig(ChoiMat)
    # take transpose of Smat so we are getting eigenvectors; 
    # (transpose of unitary matrix is still unitary)
    # see https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html
    Smat = Smat.transpose()
    
    KrausSet = []
    for idx, l in enumerate(Lambda):
        svec = Smat[idx]
        lam = math.sqrt(l)
        kvec = lam*svec
        KrausSet.append(matrize(kvec))
    return KrausSet

# From Kraus to Choi

Given a Kraus set $\{A_a: a \in \Sigma\}$, the natural representation is 

$$
    K(\Phi) = \sum_{a \in \Sigma} A_a \otimes A_a^*. 
$$

In [5]:
def Kraus_to_Nat(KrausSet):
    K = 0
    for A in KrausSet:
        K += np.kron( A, np.conjugate(A) )
    return K

# From Kraus to Stinespring

Given a Kraus set $\{A_a: a \in \Sigma\}$, the Steinspring representation is 
$$
    A = \sum_{a \in \Sigma} |a \rangle \langle a| \otimes A_a. 
$$

In [6]:
def Kraus_to_Stine(Kset):
    A = 0
    L = len(Kset)
    for idx, K in enumerate(Kset):
        T = np.zeros((L,L))
        T[idx, idx] = 1
        A += np.kron(T, K)
    return A

# From Stinespring to Kraus

Given a Stinespring dilation $A$ any Kraus operator is given as 
$$
    A_a = \text{Tr}_{\mathcal{Z}}( A (|a \rangle \langle a| \otimes \mathbb{1} ) 
$$

In [8]:
M = np.eye(4)
Choi = PMat_to_Choi(M)
Kraus = Choi_to_Kraus(Choi)
Nat = Kraus_to_Nat(Kraus)
Stine = Kraus_to_Stine(Kraus)
print("Natural Input:\n")
print(M)
print("\n\n Natural:\n")
print(Nat)
print("\n\n Choi:\n")
print(Choi)
print("\n\n Kraus:\n")
print(Kraus)
print("\n\n Stinespring:\n")
print(Stine)

Natural Input:

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


 Natural:

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


 Choi:

[[1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 1.]]


 Kraus:

[array([[1., 0.],
       [0., 1.]]), array([[-0.,  0.],
       [ 0.,  0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]])]


 Stinespring:

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
K = [[1, 0]]